# Hyperparameter Optimization (HPO) of Machine Learning Models
L. Yang and A. Shami, “On hyperparameter optimization of machine learning algorithms: Theory and practice,” Neurocomputing, vol. 415, pp. 295–316, 2020, doi: https://doi.org/10.1016/j.neucom.2020.07.061.

### **Sample code for regression problems**  
**Dataset used:**  
&nbsp; Boson Housing dataset from sklearn

**Machine learning algorithms used:**  
&nbsp; Random forest (RF), support vector machine (SVM), k-nearest neighbor (KNN), artificial neural network (ANN)

**HPO algorithms used:**  
&nbsp; Grid search, random search, hyperband, Bayesian Optimization with Gaussian Processes (BO-GP), Bayesian Optimization with Tree-structured Parzen Estimator (BO-TPE), particle swarm optimization (PSO), genetic algorithm (GA).

**Performance metric:**  
&nbsp; Mean square error (MSE)

In [6]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split,cross_val_score
from sklearn.ensemble import RandomForestClassifier,RandomForestRegressor
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.svm import SVC,SVR
from sklearn import datasets
import scipy.stats as stats

## Load Boston Housing dataset
We will take the Housing dataset which contains information about different houses in Boston. There are 506 samples and 13 feature variables in this Boston dataset. The main goal is to predict the value of prices of the house using the given features.

You can read more about the data and the variables [[1]](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html) [[2]](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_boston.html).

In [7]:
X, y = datasets.load_boston(return_X_y=True)

In [8]:
datasets.load_boston()

{'data': array([[6.3200e-03, 1.8000e+01, 2.3100e+00, ..., 1.5300e+01, 3.9690e+02,
         4.9800e+00],
        [2.7310e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9690e+02,
         9.1400e+00],
        [2.7290e-02, 0.0000e+00, 7.0700e+00, ..., 1.7800e+01, 3.9283e+02,
         4.0300e+00],
        ...,
        [6.0760e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         5.6400e+00],
        [1.0959e-01, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9345e+02,
         6.4800e+00],
        [4.7410e-02, 0.0000e+00, 1.1930e+01, ..., 2.1000e+01, 3.9690e+02,
         7.8800e+00]]),
 'target': array([24. , 21.6, 34.7, 33.4, 36.2, 28.7, 22.9, 27.1, 16.5, 18.9, 15. ,
        18.9, 21.7, 20.4, 18.2, 19.9, 23.1, 17.5, 20.2, 18.2, 13.6, 19.6,
        15.2, 14.5, 15.6, 13.9, 16.6, 14.8, 18.4, 21. , 12.7, 14.5, 13.2,
        13.1, 13.5, 18.9, 20. , 21. , 24.7, 30.8, 34.9, 26.6, 25.3, 24.7,
        21.2, 19.3, 20. , 16.6, 14.4, 19.4, 19.7, 20.5, 25. , 23.4, 18.9,
        35.4, 24.7, 3

## Baseline Machine Learning models: Regressors with Default Hyperparameters

In [9]:
#Random Forest
clf = RandomForestRegressor()
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error') # 3-fold cross-validation
print("MSE:"+ str(-scores.mean()))

MSE:29.442144369904657


In [10]:
#SVM
clf = SVR(gamma='scale')
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:77.42951812579331


In [11]:
#KNN
clf = KNeighborsRegressor()
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

MSE:81.48773186343571


In [12]:
#ANN
from keras.models import Sequential, Model
from keras.layers import Dense, Input
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasRegressor
from keras.callbacks import EarlyStopping
def ANN(optimizer = 'adam',neurons=32,batch_size=32,epochs=50,activation='relu',patience=5,loss='mse'):
    model = Sequential()
    model.add(Dense(neurons, input_shape=(X.shape[1],), activation=activation))
    model.add(Dense(neurons, activation=activation))
    model.add(Dense(1))
    model.compile(optimizer = optimizer, loss=loss)
    early_stopping = EarlyStopping(monitor="loss", patience = patience)# early stop patience
    history = model.fit(X, y,
              batch_size=batch_size,
              epochs=epochs,
              callbacks = [early_stopping],
              verbose=0) #verbose set to 1 will show the training process
    return model

In [36]:
clf = KerasRegressor(build_fn=ANN, verbose=0)
scores = cross_val_score(clf, X, y, cv=3,scoring='neg_mean_squared_error')
print("MSE:"+ str(-scores.mean()))

Epoch 1/50
16/16 [==============================] - 1s 864us/step - loss: 1278.2802
Epoch 2/50
16/16 [==============================] - 0s 931us/step - loss: 152.1980
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 91.7846
Epoch 4/50
16/16 [==============================] - 0s 997us/step - loss: 73.3953
Epoch 5/50
16/16 [==============================] - 0s 1ms/step - loss: 65.2714
Epoch 6/50
16/16 [==============================] - 0s 1ms/step - loss: 62.5714
Epoch 7/50
16/16 [==============================] - 0s 1ms/step - loss: 63.4472
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 54.3033
Epoch 9/50
16/16 [==============================] - 0s 1ms/step - loss: 62.1525
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 63.3178
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 51.8684
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 52.5990
Epoch 13/50
16/16 [=========

16/16 [==============================] - 0s 1ms/step - loss: 205.0518
Epoch 3/50
16/16 [==============================] - 0s 1ms/step - loss: 155.4403
Epoch 4/50
16/16 [==============================] - 0s 1ms/step - loss: 89.7518
Epoch 5/50
16/16 [==============================] - 0s 996us/step - loss: 69.2133
Epoch 6/50
16/16 [==============================] - 0s 931us/step - loss: 61.7153
Epoch 7/50
16/16 [==============================] - 0s 998us/step - loss: 58.6619
Epoch 8/50
16/16 [==============================] - 0s 1ms/step - loss: 60.7168
Epoch 9/50
16/16 [==============================] - 0s 997us/step - loss: 56.7078
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 55.2007
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 49.5567
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 53.7992
Epoch 13/50
16/16 [==============================] - 0s 1ms/step - loss: 50.3405
Epoch 14/50
16/16 [==================

## HPO Algorithm 1: Grid Search
Search all the given hyper-parameter configurations

**Advantages:**
* Simple implementation.  

**Disadvantages:**  
* Time-consuming,
* Only efficient with categorical HPs.

In [41]:
#Random Forest
from sklearn.model_selection import GridSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': [10, 20, 30],
    'max_features': ['sqrt',0.5],
    'max_depth': [15,20,30,50],
    'min_samples_leaf': [1,2,4,8],
    "bootstrap":[True,False],
    "criterion":['mse','mae']
}
clf = RandomForestRegressor(random_state=0)
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'bootstrap': True, 'criterion': 'mse', 'max_depth': 15, 'max_features': 0.5, 'min_samples_leaf': 4, 'n_estimators': 20}
MSE:25.639921597582575


In [44]:
#SVM
from sklearn.model_selection import GridSearchCV
rf_params = {
    'C': [1,10, 100],
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":[0.01,0.1,1]
}
clf = SVR(gamma='scale')
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'C': 100, 'epsilon': 0.01, 'kernel': 'poly'}
MSE:67.07644831331122


In [63]:
###### KNN
from sklearn.model_selection import GridSearchCV
rf_params = {
    'n_neighbors': [2,3,5,6,7,10]
}
clf = KNeighborsRegressor()
grid = GridSearchCV(clf, rf_params, cv=3, scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))

{'n_neighbors': 6}
MSE:80.83005201647829


In [64]:
#ANN
from sklearn.model_selection import GridSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32],
    'neurons':[16,32],
    'epochs':[20,50],
    'patience':[2,5]
}
clf = KerasRegressor(build_fn=ANN, verbose=0)
grid = GridSearchCV(clf, rf_params, cv=3,scoring='neg_mean_squared_error')
grid.fit(X, y)
print(grid.best_params_)
print("MSE:"+ str(-grid.best_score_))
# {'activation': 'relu', 'batch_size': 16, 'epochs': 50, 'loss': 'mse', 'neurons': 16, 'optimizer': 'adam', 'patience': 2}

Epoch 1/20
32/32 [==============================] - 0s 951us/step - loss: 252.6902
Epoch 2/20
32/32 [==============================] - 0s 804us/step - loss: 88.3157
Epoch 3/20
32/32 [==============================] - 0s 901us/step - loss: 69.4714
Epoch 4/20
32/32 [==============================] - 0s 933us/step - loss: 78.1292
Epoch 5/20
32/32 [==============================] - 0s 868us/step - loss: 73.3091
Epoch 6/20
32/32 [==============================] - 0s 837us/step - loss: 63.6567
Epoch 7/20
32/32 [==============================] - 0s 966us/step - loss: 70.9138
Epoch 8/20
32/32 [==============================] - 0s 997us/step - loss: 63.3938
Epoch 9/20
32/32 [==============================] - 0s 996us/step - loss: 61.8624
Epoch 10/20
32/32 [==============================] - 0s 998us/step - loss: 70.4802
Epoch 11/20
32/32 [==============================] - 0s 998us/step - loss: 59.8225
Epoch 12/20
32/32 [==============================] - 0s 870us/step - loss: 55.1166
Epoch 13/20


32/32 [==============================] - 1s 869us/step - loss: 963.0057
Epoch 2/20
32/32 [==============================] - 0s 870us/step - loss: 231.9971
Epoch 3/20
32/32 [==============================] - 0s 899us/step - loss: 150.5670
Epoch 4/20
32/32 [==============================] - 0s 1ms/step - loss: 123.2691
Epoch 5/20
32/32 [==============================] - 0s 1ms/step - loss: 105.7064
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 81.8214
Epoch 7/20
32/32 [==============================] - 0s 1ms/step - loss: 89.6295
Epoch 8/20
32/32 [==============================] - 0s 1ms/step - loss: 88.5492
Epoch 9/20
32/32 [==============================] - 0s 869us/step - loss: 92.2292
Epoch 10/20
32/32 [==============================] - 0s 1ms/step - loss: 85.2153
Epoch 11/20
32/32 [==============================] - 0s 903us/step - loss: 69.3428
Epoch 12/20
32/32 [==============================] - 0s 901us/step - loss: 73.0309
Epoch 13/20
32/32 [============

32/32 [==============================] - 0s 1ms/step - loss: 52.9330
Epoch 12/20
32/32 [==============================] - 0s 900us/step - loss: 61.3609
Epoch 13/20
32/32 [==============================] - 0s 933us/step - loss: 52.1520
Epoch 14/20
32/32 [==============================] - 0s 997us/step - loss: 44.8304
Epoch 15/20
32/32 [==============================] - 0s 965us/step - loss: 67.3017
Epoch 16/20
32/32 [==============================] - 0s 933us/step - loss: 48.4477
Epoch 17/20
32/32 [==============================] - 0s 836us/step - loss: 54.5332
Epoch 18/20
32/32 [==============================] - 0s 869us/step - loss: 45.2717
Epoch 19/20
32/32 [==============================] - 0s 901us/step - loss: 47.6681
Epoch 20/20
32/32 [==============================] - 0s 836us/step - loss: 48.7889
Epoch 1/20
32/32 [==============================] - 1s 901us/step - loss: 33250.3198
Epoch 2/20
32/32 [==============================] - 0s 869us/step - loss: 2235.6547
Epoch 3/20
32/3

Epoch 15/20
32/32 [==============================] - 0s 1ms/step - loss: 54.1075
Epoch 16/20
32/32 [==============================] - 0s 933us/step - loss: 62.0695
Epoch 17/20
32/32 [==============================] - 0s 965us/step - loss: 44.1086
Epoch 18/20
32/32 [==============================] - 0s 965us/step - loss: 51.6762
Epoch 19/20
32/32 [==============================] - 0s 965us/step - loss: 51.9286
Epoch 20/20
32/32 [==============================] - 0s 869us/step - loss: 54.1304
Epoch 1/20
32/32 [==============================] - 0s 876us/step - loss: 15779.1517
Epoch 2/20
32/32 [==============================] - 0s 804us/step - loss: 997.7068
Epoch 3/20
32/32 [==============================] - 0s 836us/step - loss: 192.3226
Epoch 4/20
32/32 [==============================] - 0s 869us/step - loss: 110.1210
Epoch 5/20
32/32 [==============================] - 0s 868us/step - loss: 86.0554
Epoch 6/20
32/32 [==============================] - 0s 901us/step - loss: 84.2839
Epoch 

32/32 [==============================] - 0s 869us/step - loss: 84.7907
Epoch 3/20
32/32 [==============================] - 0s 997us/step - loss: 82.6274
Epoch 4/20
32/32 [==============================] - 0s 965us/step - loss: 64.4309
Epoch 5/20
32/32 [==============================] - 0s 932us/step - loss: 52.5224
Epoch 6/20
32/32 [==============================] - 0s 1ms/step - loss: 68.4038
Epoch 7/20
32/32 [==============================] - 0s 933us/step - loss: 54.7922
Epoch 8/20
32/32 [==============================] - 0s 2ms/step - loss: 62.3880
Epoch 9/20
32/32 [==============================] - 0s 1ms/step - loss: 68.2052
Epoch 10/20
32/32 [==============================] - 0s 805us/step - loss: 69.6521
Epoch 11/20
32/32 [==============================] - 0s 966us/step - loss: 57.3888
Epoch 12/20
32/32 [==============================] - 0s 966us/step - loss: 58.5051
Epoch 1/20
32/32 [==============================] - 1s 1ms/step - loss: 6380.0729
Epoch 2/20
32/32 [============

...

16/16 [==============================] - 0s 1ms/step - loss: 9.3480
Epoch 18/50
16/16 [==============================] - 0s 1ms/step - loss: 8.7095
Epoch 19/50
16/16 [==============================] - 0s 1ms/step - loss: 8.4179
Epoch 20/50
16/16 [==============================] - 0s 1ms/step - loss: 8.0667
Epoch 21/50
16/16 [==============================] - 0s 1ms/step - loss: 7.9398
Epoch 22/50
16/16 [==============================] - 0s 1ms/step - loss: 7.8977
Epoch 23/50
16/16 [==============================] - 0s 1ms/step - loss: 7.4212
Epoch 24/50
16/16 [==============================] - 0s 1ms/step - loss: 8.0612
Epoch 25/50
16/16 [==============================] - 0s 1ms/step - loss: 7.0256
Epoch 26/50
16/16 [==============================] - 0s 1ms/step - loss: 6.8291
Epoch 27/50
16/16 [==============================] - 0s 997us/step - loss: 7.0131
Epoch 28/50
16/16 [==============================] - 0s 1ms/step - loss: 6.9623
Epoch 29/50
16/16 [==============================]

16/16 [==============================] - 0s 997us/step - loss: 7.8319
Epoch 20/50
16/16 [==============================] - 0s 1ms/step - loss: 7.3345
Epoch 21/50
16/16 [==============================] - 0s 1ms/step - loss: 7.1434
Epoch 22/50
16/16 [==============================] - 0s 1ms/step - loss: 6.8589
Epoch 23/50
16/16 [==============================] - 0s 1ms/step - loss: 6.2423
Epoch 24/50
16/16 [==============================] - 0s 1ms/step - loss: 6.6908
Epoch 25/50
16/16 [==============================] - 0s 1ms/step - loss: 6.6084
Epoch 26/50
16/16 [==============================] - 0s 1ms/step - loss: 6.2676
Epoch 27/50
16/16 [==============================] - 0s 1ms/step - loss: 6.0207
Epoch 28/50
16/16 [==============================] - 0s 1ms/step - loss: 6.1571
Epoch 29/50
16/16 [==============================] - 0s 1ms/step - loss: 5.6302
Epoch 30/50
16/16 [==============================] - 0s 1ms/step - loss: 6.4640
Epoch 31/50
16/16 [==============================]

16/16 [==============================] - 0s 931us/step - loss: 6.4382
Epoch 27/50
16/16 [==============================] - 0s 997us/step - loss: 6.2630
Epoch 28/50
16/16 [==============================] - 0s 1ms/step - loss: 5.7787
Epoch 29/50
16/16 [==============================] - 0s 1ms/step - loss: 5.6658
Epoch 30/50
16/16 [==============================] - 0s 1ms/step - loss: 5.5733
Epoch 31/50
16/16 [==============================] - 0s 1ms/step - loss: 5.7115
Epoch 32/50
16/16 [==============================] - 0s 1ms/step - loss: 5.9522
Epoch 33/50
16/16 [==============================] - 0s 1ms/step - loss: 5.8723
Epoch 34/50
16/16 [==============================] - 0s 997us/step - loss: 5.6234
Epoch 35/50
16/16 [==============================] - 0s 997us/step - loss: 5.2513
Epoch 36/50
16/16 [==============================] - 0s 1ms/step - loss: 5.7870
Epoch 37/50
16/16 [==============================] - 0s 1ms/step - loss: 5.7935
Epoch 38/50
16/16 [=========================

16/16 [==============================] - 0s 1ms/step - loss: 6.0881
Epoch 28/50
16/16 [==============================] - 0s 997us/step - loss: 5.9071
Epoch 29/50
16/16 [==============================] - 0s 1ms/step - loss: 6.1194
Epoch 30/50
16/16 [==============================] - 0s 1ms/step - loss: 6.1693
Epoch 31/50
16/16 [==============================] - 0s 997us/step - loss: 5.6928
Epoch 32/50
16/16 [==============================] - 0s 1ms/step - loss: 5.9537
Epoch 33/50
16/16 [==============================] - 0s 1ms/step - loss: 5.9883
Epoch 34/50
16/16 [==============================] - 0s 1ms/step - loss: 6.0626
Epoch 35/50
16/16 [==============================] - 0s 1ms/step - loss: 6.0543
Epoch 36/50
16/16 [==============================] - 0s 1ms/step - loss: 6.3725
Epoch 37/50
16/16 [==============================] - 0s 931us/step - loss: 5.6158
Epoch 38/50
16/16 [==============================] - 0s 997us/step - loss: 5.9518
Epoch 39/50
16/16 [=========================

32/32 [==============================] - 0s 997us/step - loss: 64.5540
Epoch 29/50
32/32 [==============================] - 0s 901us/step - loss: 55.5256
Epoch 30/50
32/32 [==============================] - 0s 997us/step - loss: 62.4233
Epoch 31/50
32/32 [==============================] - 0s 901us/step - loss: 56.8894
Epoch 32/50
32/32 [==============================] - 0s 869us/step - loss: 61.1999
Epoch 33/50
32/32 [==============================] - 0s 869us/step - loss: 65.1801
Epoch 34/50
32/32 [==============================] - 0s 771us/step - loss: 56.7656
Epoch 35/50
32/32 [==============================] - 0s 772us/step - loss: 57.9706
Epoch 36/50
32/32 [==============================] - 0s 966us/step - loss: 54.1350
Epoch 37/50
32/32 [==============================] - 0s 901us/step - loss: 49.8870
Epoch 38/50
32/32 [==============================] - 0s 933us/step - loss: 52.4120
Epoch 39/50
32/32 [==============================] - 0s 869us/step - loss: 63.0401
Epoch 40/50
32/3

## HPO Algorithm 2: Random Search
Randomly search hyper-parameter combinations in the search space

**Advantages:**
* More efficient than GS.
* Enable parallelization. 

**Disadvantages:**  
* Not consider previous results.
* Not efficient with conditional HPs.

In [72]:
#Random Forest
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,13),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    "criterion":['mse']
}
n_iter_search=20 #number of iterations is set to 20, you can increase this number if time permits
clf = RandomForestRegressor(random_state=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'criterion': 'mse', 'max_depth': 23, 'max_features': 5, 'min_samples_leaf': 4, 'min_samples_split': 2, 'n_estimators': 20}
MSE:26.051461529434523


In [85]:
#SVM
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['poly'],
    "epsilon":stats.uniform(0,1)
}
n_iter_search=20
clf = SVR(gamma='scale')
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'C': 42.69069287566832, 'epsilon': 0.18938158548855621, 'kernel': 'poly'}
MSE:59.90776994341321


In [119]:
#KNN
from scipy.stats import randint as sp_randint
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'n_neighbors': sp_randint(1,20),
}
n_iter_search=15
clf = KNeighborsRegressor()
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

{'n_neighbors': 13}
MSE:80.74121499347262


In [120]:
#ANN
from scipy.stats import randint as sp_randint
from random import randrange as sp_randrange
from sklearn.model_selection import RandomizedSearchCV
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32,64],
    'neurons':sp_randint(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':sp_randint(3,20)
}
n_iter_search=10
clf = KerasRegressor(build_fn=ANN, verbose=0)
Random = RandomizedSearchCV(clf, param_distributions=rf_params,n_iter=n_iter_search,cv=3,scoring='neg_mean_squared_error')
Random.fit(X, y)
print(Random.best_params_)
print("MSE:"+ str(-Random.best_score_))

Epoch 1/20
16/16 [==============================] - 1s 1ms/step - loss: 20.0468
Epoch 2/20
16/16 [==============================] - 0s 1ms/step - loss: 13.6247
Epoch 3/20
16/16 [==============================] - 0s 1ms/step - loss: 10.4694
Epoch 4/20
16/16 [==============================] - 0s 1ms/step - loss: 9.0317
Epoch 5/20
16/16 [==============================] - 0s 1ms/step - loss: 8.6338
Epoch 6/20
16/16 [==============================] - 0s 1ms/step - loss: 8.1070
Epoch 7/20
16/16 [==============================] - 0s 1ms/step - loss: 7.2633
Epoch 8/20
16/16 [==============================] - 0s 1ms/step - loss: 6.5888
Epoch 9/20
16/16 [==============================] - 0s 1ms/step - loss: 6.6713
Epoch 10/20
16/16 [==============================] - 0s 1ms/step - loss: 6.0566
Epoch 11/20
16/16 [==============================] - 0s 1ms/step - loss: 5.9426
Epoch 12/20
16/16 [==============================] - 0s 1ms/step - loss: 5.6008
Epoch 13/20
16/16 [===========================

32/32 [==============================] - 1s 1ms/step - loss: 193.6537
Epoch 2/50
32/32 [==============================] - 0s 1ms/step - loss: 9.8825
Epoch 3/50
32/32 [==============================] - 0s 1ms/step - loss: 6.8794
Epoch 4/50
32/32 [==============================] - 0s 1ms/step - loss: 6.9314
Epoch 5/50
32/32 [==============================] - 0s 1ms/step - loss: 6.5010
Epoch 6/50
32/32 [==============================] - 0s 1ms/step - loss: 6.2457
Epoch 7/50
32/32 [==============================] - 0s 1ms/step - loss: 5.6592
Epoch 8/50
32/32 [==============================] - 0s 1ms/step - loss: 5.8034
Epoch 9/50
32/32 [==============================] - 0s 2ms/step - loss: 6.2970
Epoch 10/50
32/32 [==============================] - 0s 1ms/step - loss: 6.5203
Epoch 11/50
32/32 [==============================] - 0s 1ms/step - loss: 6.3331
Epoch 12/50
32/32 [==============================] - 0s 1ms/step - loss: 6.3098
Epoch 13/50
32/32 [==============================] - 0s 1m

32/32 [==============================] - 0s 1ms/step - loss: 31.5119
Epoch 30/50
32/32 [==============================] - 0s 1ms/step - loss: 36.3912
Epoch 31/50
32/32 [==============================] - 0s 1ms/step - loss: 30.5109
Epoch 32/50
32/32 [==============================] - 0s 1ms/step - loss: 29.6640
Epoch 33/50
32/32 [==============================] - 0s 1ms/step - loss: 31.5039
Epoch 34/50
32/32 [==============================] - 0s 1ms/step - loss: 25.5402
Epoch 35/50
32/32 [==============================] - 0s 1ms/step - loss: 34.6157
Epoch 36/50
32/32 [==============================] - 0s 1ms/step - loss: 28.3847
Epoch 37/50
32/32 [==============================] - 0s 965us/step - loss: 29.4796
Epoch 38/50
32/32 [==============================] - 0s 1ms/step - loss: 24.7606
Epoch 39/50
32/32 [==============================] - 0s 1ms/step - loss: 25.5475
Epoch 40/50
32/32 [==============================] - 0s 997us/step - loss: 25.5161
Epoch 41/50
32/32 [=================

32/32 [==============================] - 0s 1ms/step - loss: 55.2662
Epoch 30/50
32/32 [==============================] - 0s 997us/step - loss: 58.7069
Epoch 31/50
32/32 [==============================] - 0s 965us/step - loss: 49.4369
Epoch 32/50
32/32 [==============================] - 0s 1ms/step - loss: 50.8146
Epoch 33/50
32/32 [==============================] - 0s 965us/step - loss: 55.1149
Epoch 34/50
32/32 [==============================] - 0s 934us/step - loss: 48.7040
Epoch 35/50
32/32 [==============================] - 0s 901us/step - loss: 56.8207
Epoch 36/50
32/32 [==============================] - 0s 965us/step - loss: 47.9625
Epoch 37/50
32/32 [==============================] - 0s 1ms/step - loss: 51.9670
Epoch 38/50
32/32 [==============================] - 0s 933us/step - loss: 49.1390
Epoch 39/50
32/32 [==============================] - 0s 901us/step - loss: 50.8135
Epoch 40/50
32/32 [==============================] - 0s 901us/step - loss: 46.8254
Epoch 41/50
32/32 [===

16/16 [==============================] - 0s 1ms/step - loss: 97.7594
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 84.3029
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 100.1025
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 79.7683
Epoch 13/50
16/16 [==============================] - 0s 1ms/step - loss: 82.2763
Epoch 14/50
16/16 [==============================] - 0s 1ms/step - loss: 83.6444
Epoch 15/50
16/16 [==============================] - 0s 997us/step - loss: 88.6131
Epoch 16/50
16/16 [==============================] - 0s 1ms/step - loss: 65.3462
Epoch 17/50
16/16 [==============================] - 0s 1ms/step - loss: 83.3445
Epoch 18/50
16/16 [==============================] - 0s 1ms/step - loss: 66.1922
Epoch 19/50
16/16 [==============================] - 0s 864us/step - loss: 63.1686
Epoch 20/50
16/16 [==============================] - 0s 928us/step - loss: 54.0320
Epoch 21/50
16/16 [==============

32/32 [==============================] - 0s 1ms/step - loss: 65.1665
Epoch 10/20
32/32 [==============================] - 0s 979us/step - loss: 100.0570
Epoch 11/20
32/32 [==============================] - 0s 901us/step - loss: 57.4627
Epoch 12/20
32/32 [==============================] - 0s 901us/step - loss: 77.6138
Epoch 13/20
32/32 [==============================] - 0s 1ms/step - loss: 73.3158
Epoch 14/20
32/32 [==============================] - 0s 1ms/step - loss: 62.3215
Epoch 15/20
32/32 [==============================] - 0s 2ms/step - loss: 55.1408
Epoch 16/20
32/32 [==============================] - 0s 1ms/step - loss: 85.7716
Epoch 17/20
32/32 [==============================] - 0s 1ms/step - loss: 43.9775
Epoch 18/20
32/32 [==============================] - 0s 2ms/step - loss: 52.1985
Epoch 19/20
32/32 [==============================] - 0s 1ms/step - loss: 58.8890
Epoch 20/20
32/32 [==============================] - 0s 1ms/step - loss: 65.2407
Epoch 1/20
32/32 [===============

16/16 [==============================] - 0s 1ms/step - loss: 7.2251
Epoch 10/50
16/16 [==============================] - 0s 1ms/step - loss: 7.7301
Epoch 11/50
16/16 [==============================] - 0s 1ms/step - loss: 7.6088
Epoch 12/50
16/16 [==============================] - 0s 1ms/step - loss: 7.6353
Epoch 13/50
16/16 [==============================] - 0s 1ms/step - loss: 7.0995
Epoch 14/50
16/16 [==============================] - 0s 1ms/step - loss: 6.9095
Epoch 15/50
16/16 [==============================] - 0s 1ms/step - loss: 7.0441
Epoch 16/50
16/16 [==============================] - 0s 1ms/step - loss: 6.0288
Epoch 17/50
16/16 [==============================] - 0s 1ms/step - loss: 6.4304
Epoch 18/50
16/16 [==============================] - 0s 1ms/step - loss: 6.0709
Epoch 19/50
16/16 [==============================] - 0s 1ms/step - loss: 5.7324
Epoch 1/20
32/32 [==============================] - 0s 901us/step - loss: 20.5023
Epoch 2/20
32/32 [==============================] 

32/32 [==============================] - 0s 868us/step - loss: 4.8160
Epoch 31/50
32/32 [==============================] - 0s 933us/step - loss: 4.8008
Epoch 32/50
32/32 [==============================] - 0s 836us/step - loss: 5.0948
Epoch 33/50
32/32 [==============================] - 0s 901us/step - loss: 5.2794
Epoch 34/50
32/32 [==============================] - 0s 836us/step - loss: 4.7240
Epoch 35/50
32/32 [==============================] - 0s 869us/step - loss: 4.6796
Epoch 36/50
32/32 [==============================] - 0s 838us/step - loss: 4.8887
Epoch 37/50
32/32 [==============================] - 0s 836us/step - loss: 4.2565
Epoch 38/50
32/32 [==============================] - 0s 836us/step - loss: 4.2674
Epoch 39/50
32/32 [==============================] - 0s 836us/step - loss: 4.5730
Epoch 40/50
32/32 [==============================] - 0s 837us/step - loss: 4.5428
Epoch 41/50
32/32 [==============================] - 0s 966us/step - loss: 4.4371
Epoch 42/50
32/32 [=========

32/32 [==============================] - 0s 966us/step - loss: 5.3072
Epoch 31/50
32/32 [==============================] - 0s 997us/step - loss: 4.6022
Epoch 32/50
32/32 [==============================] - 0s 997us/step - loss: 5.2229
Epoch 33/50
32/32 [==============================] - 0s 869us/step - loss: 5.0291
Epoch 34/50
32/32 [==============================] - 0s 836us/step - loss: 4.9063
Epoch 35/50
32/32 [==============================] - 0s 836us/step - loss: 5.0494
Epoch 36/50
32/32 [==============================] - 0s 837us/step - loss: 4.8552
Epoch 37/50
32/32 [==============================] - 0s 901us/step - loss: 4.4857
Epoch 38/50
32/32 [==============================] - 0s 901us/step - loss: 4.9379
Epoch 39/50
32/32 [==============================] - 0s 933us/step - loss: 5.0961
Epoch 40/50
32/32 [==============================] - 0s 934us/step - loss: 5.0730
Epoch 41/50
32/32 [==============================] - 0s 901us/step - loss: 5.1056
Epoch 1/20
8/8 [============

Epoch 32/50
32/32 [==============================] - 0s 1ms/step - loss: 33.5408
Epoch 33/50
32/32 [==============================] - 0s 997us/step - loss: 29.4710
Epoch 34/50
32/32 [==============================] - 0s 1ms/step - loss: 42.7682
Epoch 35/50
32/32 [==============================] - 0s 1ms/step - loss: 27.7903
Epoch 36/50
32/32 [==============================] - 0s 1ms/step - loss: 30.8250
Epoch 37/50
32/32 [==============================] - 0s 1ms/step - loss: 32.4584
Epoch 38/50
32/32 [==============================] - 0s 1ms/step - loss: 29.8937
Epoch 39/50
32/32 [==============================] - 0s 1ms/step - loss: 38.7952
Epoch 40/50
32/32 [==============================] - 0s 1ms/step - loss: 29.0380
Epoch 41/50
32/32 [==============================] - 0s 1ms/step - loss: 30.2144
Epoch 42/50
32/32 [==============================] - 0s 1ms/step - loss: 28.1664
Epoch 43/50
32/32 [==============================] - 0s 1ms/step - loss: 28.7404
Epoch 44/50
32/32 [=======

## HPO Algorithm 3: Hyperband
Generate small-sized subsets and allocate budgets to each hyper-parameter combination based on its performance

**Advantages:**
* Enable parallelization.  

**Disadvantages:**  
* Not efficient with conditional HPs.
* Require subsets with small budgets to be representative.

In [20]:
#Random Forest
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': sp_randint(10,100),
    "max_features":sp_randint(1,13),
    'max_depth': sp_randint(5,50),
    "min_samples_split":sp_randint(2,11),
    "min_samples_leaf":sp_randint(1,11),
    "criterion":['mse','mae']
}
clf = RandomForestRegressor(random_state=0)
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=10,max_iter=100,scoring='neg_mean_squared_error')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

ModuleNotFoundError: No module named 'hyperband'

In [14]:
#SVM
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
rf_params = {
    'C': stats.uniform(0,50),
    "kernel":['poly','rbf','sigmoid'],
    "epsilon":stats.uniform(0,1)
}
clf = SVR(gamma='scale')
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=1,max_iter=10,scoring='neg_mean_squared_error',resource_param='C')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

ModuleNotFoundError: No module named 'hyperband'

In [42]:
#KNN
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
rf_params = {
    'n_neighbors': range(1,20),
}
clf = KNeighborsRegressor()
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=1,max_iter=20,scoring='neg_mean_squared_error',resource_param='n_neighbors')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

{'n_neighbors': 6}
MSE:80.87024044795783


In [9]:
#ANN
from hyperband import HyperbandSearchCV
from scipy.stats import randint as sp_randint
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32,64],
    'neurons':sp_randint(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':sp_randint(3,20)
}
clf = KerasRegressor(build_fn=ANN, epochs=20, verbose=0)
hyper = HyperbandSearchCV(clf, param_distributions =rf_params,cv=3,min_iter=1,max_iter=10,scoring='neg_mean_squared_error',resource_param='epochs')
hyper.fit(X, y)
print(hyper.best_params_)
print("MSE:"+ str(-hyper.best_score_))

{'patience': 3, 'batch_size': 16, 'loss': 'mse', 'activation': 'relu', 'optimizer': 'adam', 'neurons': 48, 'epochs': 10}
MSE:56.59321886927081


## HPO Algorithm 4: BO-GP
Bayesian Optimization with Gaussian Process (BO-GP)

**Advantages:**
* Fast convergence speed for continuous HPs.  

**Disadvantages:**  
* Poor capacity for parallelization.
* Not efficient with conditional HPs.

### Using skopt.BayesSearchCV

In [21]:
#Random Forest
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': Integer(10,100),
    "max_features":Integer(1,13),
    'max_depth': Integer(5,50),
    "min_samples_split":Integer(2,11),
    "min_samples_leaf":Integer(1,11),
    "criterion":['mse','mae']
}
clf = RandomForestRegressor(random_state=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, scoring='neg_mean_squared_error') 
#number of iterations is set to 20, you can increase this number if time permits
Bayes.fit(X, y)
print(Bayes.best_params_)
bclf = Bayes.best_estimator_
print("MSE:"+ str(-Bayes.best_score_))

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


OrderedDict([('criterion', 'mse'), ('max_depth', 39), ('max_features', 9), ('min_samples_leaf', 2), ('min_samples_split', 2), ('n_estimators', 100)])
MSE:27.230832227590795


In [22]:
#SVM
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'C': Real(0,50),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': Real(0,1)
}
clf = SVR(gamma='scale')
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=20, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\Anaconda3\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
C:\ProgramData\

OrderedDict([('C', 50.0), ('epsilon', 0.0), ('kernel', 'poly')])
MSE:59.208216196430655


In [23]:
#KNN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'n_neighbors': Integer(1,20),
}
clf = KNeighborsRegressor()
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=10, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

OrderedDict([('n_neighbors', 13)])
MSE:80.7723025469514


In [24]:
#ANN
from skopt import Optimizer
from skopt import BayesSearchCV 
from skopt.space import Real, Categorical, Integer
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32,64],
    'neurons':Integer(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':Integer(3,20)
}
clf = KerasRegressor(build_fn=ANN, verbose=0)
Bayes = BayesSearchCV(clf, rf_params,cv=3,n_iter=10, scoring='neg_mean_squared_error')
Bayes.fit(X, y)
print(Bayes.best_params_)
print("MSE:"+ str(-Bayes.best_score_))

OrderedDict([('activation', 'tanh'), ('batch_size', 32), ('epochs', 40), ('loss', 'mae'), ('neurons', 63), ('optimizer', 'rmsprop'), ('patience', 10)])
MSE:50.528691798768165


### Using skopt.gp_minimize

In [25]:
#Random Forest
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = RandomForestRegressor()
# Define the hyperparameter configuration space
space  = [Integer(10, 100, name='n_estimators'),
            Integer(5, 50, name='max_depth'),
          Integer(1, 13, name='max_features'),
          Integer(2, 11, name='min_samples_split'),
          Integer(1, 11, name='min_samples_leaf'),
         Categorical(['mse', 'mae'], name='criterion')
         ]
# Define the objective function
@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
#number of iterations is set to 20, you can increase this number if time permits
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:22.6149
[10, 50, 7, 4, 5, 'mse']


In [27]:
#SVM
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = SVR(gamma='scale')
space  = [Real(0, 50, name='C'),
          Categorical(['poly','rbf','sigmoid'], name='kernel'),
          Real(0, 1, name='epsilon'),
         ]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=20, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [28]:
#KNN
from skopt.space import Real, Integer
from skopt.utils import use_named_args

reg = KNeighborsRegressor()
space  = [Integer(1, 20, name='n_neighbors')]

@use_named_args(space)
def objective(**params):
    reg.set_params(**params)

    return -np.mean(cross_val_score(reg, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
from skopt import gp_minimize
res_gp = gp_minimize(objective, space, n_calls=10, random_state=0)
print("MSE:%.4f" % res_gp.fun)
print(res_gp.x)

MSE:80.7412
[13]


## HPO Algorithm 5: BO-TPE
Bayesian Optimization with Tree-structured Parzen Estimator (TPE)

**Advantages:**
* Efficient with all types of HPs.
* Keep conditional dependencies.

**Disadvantages:**  
* Poor capacity for parallelization.

In [29]:
#Random Forest
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
# Define the objective function
def objective(params):
    params = {
        'n_estimators': int(params['n_estimators']), 
        'max_depth': int(params['max_depth']),
        'max_features': int(params['max_features']),
        "min_samples_split":int(params['min_samples_split']),
        "min_samples_leaf":int(params['min_samples_leaf']),
        "criterion":str(params['criterion'])
    }
    clf = RandomForestRegressor( **params)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

    return {'loss':score, 'status': STATUS_OK }
# Define the hyperparameter configuration space
space = {
    'n_estimators': hp.quniform('n_estimators', 10, 100, 1),
    'max_depth': hp.quniform('max_depth', 5, 50, 1),
    "max_features":hp.quniform('max_features', 1, 13, 1),
    "min_samples_split":hp.quniform('min_samples_split',2,11,1),
    "min_samples_leaf":hp.quniform('min_samples_leaf',1,11,1),
    "criterion":hp.choice('criterion',['mse','mae'])
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("Random Forest: Hyperopt estimated optimum {}".format(best))

ModuleNotFoundError: No module named 'hyperopt'

In [27]:
#SVM
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'C': abs(float(params['C'])), 
        "kernel":str(params['kernel']),
        'epsilon': abs(float(params['epsilon'])),
    }
    clf = SVR(gamma='scale', **params)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    
    return {'loss':score, 'status': STATUS_OK }

space = {
    'C': hp.normal('C', 0, 50),
    "kernel":hp.choice('kernel',['poly','rbf','sigmoid']),
    'epsilon': hp.normal('epsilon', 0, 1),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=20)
print("SVM: Hyperopt estimated optimum {}".format(best))

100%|████████████████████████████████████████████████████| 20/20 [00:03<00:00,  5.10it/s, best loss: 63.07694330646493]
SVM: Hyperopt estimated optimum {'kernel': 0, 'epsilon': 0.6784908715313539, 'C': 50.9909224663369}


In [28]:
#KNN
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        'n_neighbors': abs(int(params['n_neighbors']))
    }
    clf = KNeighborsRegressor( **params)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

    return {'loss':score, 'status': STATUS_OK }

space = {
    'n_neighbors': hp.quniform('n_neighbors', 1, 20, 1),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("KNN: Hyperopt estimated optimum {}".format(best))

100%|███████████████████████████████████████████████████| 10/10 [00:00<00:00, 125.34it/s, best loss: 81.26511555604914]
KNN: Hyperopt estimated optimum {'n_neighbors': 14.0}


In [26]:
#ANN
from hyperopt import hp, fmin, tpe, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score, StratifiedKFold
def objective(params):
    params = {
        "optimizer":str(params['optimizer']),
        "activation":str(params['activation']),
        "loss":str(params['loss']),
        'batch_size': abs(int(params['batch_size'])),
        'neurons': abs(int(params['neurons'])),
        'epochs': abs(int(params['epochs'])),
        'patience': abs(int(params['patience']))
    }
    clf = KerasRegressor(build_fn=ANN,**params, verbose=0)
    score = -np.mean(cross_val_score(clf, X, y, cv=3, 
                                    scoring="neg_mean_squared_error"))

    return {'loss':score, 'status': STATUS_OK }

space = {
    "optimizer":hp.choice('optimizer',['adam','rmsprop']),
    "activation":hp.choice('activation',['relu','tanh']),
    "loss":hp.choice('loss',['mse','mae']),
    'batch_size': hp.quniform('batch_size', 16, 64, 16),
    'neurons': hp.quniform('neurons', 10, 100, 10),
    'epochs': hp.quniform('epochs', 20, 50, 10),
    'patience': hp.quniform('patience', 3, 20, 3),
}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=10)
print("ANN: Hyperopt estimated optimum {}".format(best))

100%|████████████████████████████████████████████████████| 10/10 [04:50<00:00, 29.06s/it, best loss: 58.39425089889795]
ANN: Hyperopt estimated optimum {'activation': 1, 'patience': 18.0, 'neurons': 80.0, 'batch_size': 32.0, 'loss': 0, 'epochs': 50.0, 'optimizer': 1}


## HPO Algorithm 6: PSO
Partical swarm optimization (PSO): Each particle in a swarm communicates with other particles to detect and update the current global optimum in each iteration until the final optimum is detected.

**Advantages:**
* Efficient with all types of HPs.
* Enable parallelization. 

**Disadvantages:**  
* Require proper initialization.

In [61]:
#Random Forest
import optunity
import optunity.metrics
# Define the hyperparameter configuration space
search = {
    'n_estimators': [10, 100],
    'max_features': [1, 13],
    'max_depth': [5,50],
    "min_samples_split":[2,11],
    "min_samples_leaf":[1,11],
         }
# Define the objective function
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_estimators=None, max_features=None,max_depth=None,min_samples_split=None,min_samples_leaf=None):
    # fit the model
    model = RandomForestRegressor(n_estimators=int(n_estimators),
                                   max_features=int(max_features),
                                   max_depth=int(max_depth),
                                   min_samples_split=int(min_samples_split),
                                   min_samples_leaf=int(min_samples_leaf),
                                  )
    scores=-np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))
    return scores

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'max_depth': 37.745028056718674, 'min_samples_leaf': 1.2694935979785371, 'min_samples_split': 6.3674769629093, 'n_estimators': 97.73792799268917, 'max_features': 6.383593749999999}
MSE:26.072547695737644


In [62]:
#SVM
import optunity
import optunity.metrics
search = {
    'C': (0,50),
    'kernel':[0,3],
    'epsilon': (0, 1)
         }
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,C=None,kernel=None,epsilon=None):
    # fit the model
    if kernel<1:
        ke='poly'
    elif kernel<2:
        ke='rbf'
    else:
        ke='sigmoid'
    model = SVR(C=float(C),
                kernel=ke,
                gamma='scale',
                epsilon=float(epsilon)
                                  )

    scores=-np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

    return scores

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'kernel': 0.4658203125, 'epsilon': 0.4736328125, 'C': 25.341796875}
MSE:60.25879498950017


In [63]:
#KNN
import optunity
import optunity.metrics
search = {
    'n_neighbors': [1, 20],
         }
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,n_neighbors=None):
    # fit the model
    model = KNeighborsRegressor(n_neighbors=int(n_neighbors),
                                  )

    scores=-np.mean(cross_val_score(model, X, y, cv=3, n_jobs=-1,
                                    scoring="neg_mean_squared_error"))

    return scores

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=10,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'n_neighbors': 13.84912109375}
MSE:80.74121499347262


In [24]:
#ANN
import optunity
import optunity.metrics
search = {
    'optimizer':[0,2],
    'activation':[0,2],
    'loss':[0,2],
    'batch_size': [0, 2],
    'neurons': [10, 100],
    'epochs': [20, 50],
    'patience': [3, 20],
         }
@optunity.cross_validated(x=X, y=y, num_folds=3)
def performance(x_train, y_train, x_test, y_test,optimizer=None,activation=None,loss=None,batch_size=None,neurons=None,epochs=None,patience=None):
    # fit the model
    if optimizer<1:
        op='adam'
    else:
        op='rmsprop'
    if activation<1:
        ac='relu'
    else:
        ac='tanh'
    if loss<1:
        lo='mse'
    else:
        lo='mae'
    if batch_size<1:
        ba=16
    else:
        ba=32
    model = ANN(optimizer=op,
                activation=ac,
                loss=lo,
                batch_size=ba,
                neurons=int(neurons),
                epochs=int(epochs),
                patience=int(patience)
                                  )
    clf = KerasRegressor(build_fn=ANN, verbose=0)
    scores=-np.mean(cross_val_score(clf, X, y, cv=3, 
                                    scoring="neg_mean_squared_error"))

    return scores

optimal_configuration, info, _ = optunity.minimize(performance,
                                                  solver_name='particle swarm',
                                                  num_evals=20,
                                                   **search
                                                  )
print(optimal_configuration)
print("MSE:"+ str(info.optimum))

{'activation': 0.28555402710064154, 'neurons': 90.55370287990607, 'loss': 1.3789121219831286, 'batch_size': 0.6972214016007146, 'patience': 14.611429743835629, 'epochs': 35.81883196486749, 'optimizer': 1.1613324876783653}
MSE:39.0195776769448


## HPO Algorithm 7: Genetic Algorithm
Genetic algorithms detect well-performing hyper-parameter combinations in each generation, and pass them to the next generation until the best-performing combination is identified.

**Advantages:**
* Efficient with all types of HPs.
* Not require good initialization.
 

**Disadvantages:**  
* Poor capacity for parallelization.

### Using DEAP

In [8]:
#Random Forest
from evolutionary_search import EvolutionaryAlgorithmSearchCV
from scipy.stats import randint as sp_randint
# Define the hyperparameter configuration space
rf_params = {
    'n_estimators': range(10,100),
    "max_features":range(1,13),
    'max_depth': range(5,50),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    "criterion":['mse','mae']
}
clf = RandomForestRegressor(random_state=0)
# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(-ga1.best_score_))

Types [1, 1, 1, 1, 1, 1] and maxint [1, 89, 44, 8, 11, 9] detected
--- Evolve in 8748000 possible combinations ---
gen	nevals	avg     	min    	max     	std    
0  	10    	-32.0014	-54.374	-27.5166	7.55455
1  	6     	-28.6069	-31.6563	-26.2084	1.44316
2  	4     	-27.7354	-29.2441	-26.2084	1.11775
3  	1     	-26.4502	-27.5166	-26.2084	0.392132
4  	2     	-26.2462	-26.5859	-26.2084	0.113246
5  	9     	-26.4494	-27.7558	-26.0528	0.488179
Best individual is: {'criterion': 'mse', 'n_estimators': 97, 'max_depth': 29, 'min_samples_split': 7, 'max_features': 8, 'min_samples_leaf': 3}
with fitness: -26.052847277193734
{'criterion': 'mse', 'n_estimators': 97, 'max_depth': 29, 'min_samples_split': 7, 'max_features': 8, 'min_samples_leaf': 3}
MSE:26.052847277193734


In [13]:
#SVM
from evolutionary_search import EvolutionaryAlgorithmSearchCV
rf_params = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': np.random.uniform(0,1,100),
}
clf = SVR(gamma='scale')
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(-ga1.best_score_))

Types [2, 2, 1] and maxint [999, 99, 2] detected
--- Evolve in 300000 possible combinations ---
gen	nevals	avg     	min     	max     	std    
0  	10    	-1622.54	-7880.66	-61.3248	2655.55
1  	8     	-68.6978	-81.931 	-61.0316	8.46914
2  	5     	-64.2231	-78.2337	-60.2699	5.09324
3  	2     	-61.5322	-65.247 	-60.2699	1.32565
4  	10    	-60.9099	-62.4522	-59.4154	0.848694
5  	6     	-60.1359	-61.5717	-59.4154	0.689567
Best individual is: {'C': 36.961021855583446, 'epsilon': 0.1079957723914794, 'kernel': 'poly'}
with fitness: -59.41536985706522
{'C': 36.961021855583446, 'epsilon': 0.1079957723914794, 'kernel': 'poly'}
MSE:59.41536985706522


In [14]:
#KNN
from evolutionary_search import EvolutionaryAlgorithmSearchCV
rf_params = {
    'n_neighbors': range(1,20),
}
clf = KNeighborsRegressor()
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(-ga1.best_score_))

Types [1] and maxint [18] detected
--- Evolve in 19 possible combinations ---
gen	nevals	avg     	min     	max     	std     
0  	10    	-82.2005	-82.7914	-80.8702	0.621814
1  	7     	-81.5525	-82.7021	-80.8702	0.599417
2  	8     	-80.8702	-80.8702	-80.8702	0       
3  	8     	-80.8702	-80.8702	-80.8702	0       
4  	10    	-80.9662	-81.8295	-80.8702	0.287764
5  	8     	-80.8604	-80.8702	-80.7723	0.0293814
Best individual is: {'n_neighbors': 13}
with fitness: -80.7723025469514
{'n_neighbors': 13}
MSE:80.7723025469514


In [20]:
#ANN
from evolutionary_search import EvolutionaryAlgorithmSearchCV
# Define the hyperparameter configuration space
rf_params = {
    'optimizer': ['adam','rmsprop'],
    'activation': ['relu','tanh'],
    'loss': ['mse','mae'],
    'batch_size': [16,32,64],
    'neurons':range(10,100),
    'epochs':[20,50],
    #'epochs':[20,50,100,200],
    'patience':range(3,20)
}
clf = KerasRegressor(build_fn=ANN, verbose=0)
# Set the hyperparameters of GA    
ga1 = EvolutionaryAlgorithmSearchCV(estimator=clf,
                                   params=rf_params,
                                   scoring="neg_mean_squared_error",
                                   cv=3,
                                   verbose=1,
                                   population_size=10,
                                   gene_mutation_prob=0.10,
                                   gene_crossover_prob=0.5,
                                   tournament_size=3,
                                   generations_number=5,
                                   n_jobs=1)
ga1.fit(X, y)
print(ga1.best_params_)
print("MSE:"+ str(-ga1.best_score_))

Types [1, 1, 1, 1, 1, 1, 1] and maxint [1, 16, 2, 1, 1, 89, 1] detected
--- Evolve in 73440 possible combinations ---
gen	nevals	avg     	min     	max    	std    
0  	10    	-88.4899	-157.552	-50.727	37.5649
1  	2     	-60.9958	-99.4323	-50.727	13.8181
2  	8     	-74.4071	-174.246	-50.727	42.0684
3  	7     	-59.5146	-138.603	-50.727	26.3628
4  	3     	-54.6543	-90.0003	-50.727	11.782 
5  	3     	-50.727 	-50.727 	-50.727	7.10543e-15
Best individual is: {'patience': 5, 'batch_size': 32, 'loss': 'mse', 'activation': 'relu', 'optimizer': 'rmsprop', 'neurons': 30, 'epochs': 50}
with fitness: -50.72700946933739
{'patience': 5, 'batch_size': 32, 'loss': 'mse', 'activation': 'relu', 'optimizer': 'rmsprop', 'neurons': 30, 'epochs': 50}
MSE:50.72700946933739


### Using TPOT

In [69]:
#Random Forest
from tpot import TPOTRegressor
# Define the hyperparameter configuration space
parameters = {
    'n_estimators': range(20,200),
    "max_features":range(1,13),
    'max_depth': range(10,100),
    "min_samples_split":range(2,11),
    "min_samples_leaf":range(1,11),
    #"criterion":['mse','mae']
             }
# Set the hyperparameters of GA               
ga2 = TPOTRegressor(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.ensemble.RandomForestRegressor': parameters}, 
                                 cv = 3, scoring = 'neg_mean_squared_error')
ga2.fit(X, y)

1 operators have been imported by TPOT.
Generation 1 - Current Pareto front scores:
-1	-27.250793005044518	RandomForestRegressor(input_matrix, RandomForestRegressor__max_depth=78, RandomForestRegressor__max_features=5, RandomForestRegressor__min_samples_leaf=2, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=168)
-2	-25.332016721242965	RandomForestRegressor(RandomForestRegressor(input_matrix, RandomForestRegressor__max_depth=62, RandomForestRegressor__max_features=4, RandomForestRegressor__min_samples_leaf=5, RandomForestRegressor__min_samples_split=6, RandomForestRegressor__n_estimators=34), RandomForestRegressor__max_depth=78, RandomForestRegressor__max_features=5, RandomForestRegressor__min_samples_leaf=2, RandomForestRegressor__min_samples_split=9, RandomForestRegressor__n_estimators=168)

Generation 2 - Current Pareto front scores:
-1	-27.250793005044518	RandomForestRegressor(input_matrix, RandomForestRegressor__max_depth=78, RandomForestRegressor__

TPOTRegressor(config_dict={'sklearn.ensemble.RandomForestRegressor': {'max_depth': range(10, 100), 'min_samples_leaf': range(1, 11), 'min_samples_split': range(2, 11), 'max_features': range(1, 13), 'n_estimators': range(20, 200)}},
       crossover_rate=0.1, cv=3, disable_update_check=False, early_stop=5,
       generations=3, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=5,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='neg_mean_squared_error', subsample=1.0,
       template=None, use_dask=False, verbosity=3, warm_start=False)

In [70]:
#SVM
from tpot import TPOTRegressor

parameters = {
    'C': np.random.uniform(0,50,1000),
    "kernel":['poly','rbf','sigmoid'],
    'epsilon': np.random.uniform(0,1,100),
    'gamma': ['scale']
             }
               
ga2 = TPOTRegressor(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.svm.SVR': parameters}, 
                                 cv = 3, scoring = 'neg_mean_squared_error')
ga2.fit(X, y)

1 operators have been imported by TPOT.
Generation 1 - Current Pareto front scores:
-1	-76.22342756605103	SVR(CombineDFs(input_matrix, input_matrix), SVR__C=9.66289149097888, SVR__epsilon=0.6491076078954555, SVR__gamma=scale, SVR__kernel=rbf)
-2	-61.22653072468754	SVR(SVR(CombineDFs(input_matrix, input_matrix), SVR__C=2.988848947755768, SVR__epsilon=0.31470079088508107, SVR__gamma=scale, SVR__kernel=poly), SVR__C=49.38466344757709, SVR__epsilon=0.9554593389709706, SVR__gamma=scale, SVR__kernel=poly)

Generation 2 - Current Pareto front scores:
-1	-73.46312983258913	SVR(input_matrix, SVR__C=9.66289149097888, SVR__epsilon=0.6491076078954555, SVR__gamma=scale, SVR__kernel=poly)
-2	-61.22653072468754	SVR(SVR(CombineDFs(input_matrix, input_matrix), SVR__C=2.988848947755768, SVR__epsilon=0.31470079088508107, SVR__gamma=scale, SVR__kernel=poly), SVR__C=49.38466344757709, SVR__epsilon=0.9554593389709706, SVR__gamma=scale, SVR__kernel=poly)

Generation 3 - Current Pareto front scores:
-1	-59.46

TPOTRegressor(config_dict={'sklearn.svm.SVR': {'kernel': ['poly', 'rbf', 'sigmoid'], 'gamma': ['scale'], 'epsilon': array([0.25137, 0.0217 , ..., 0.7522 , 0.4685 ]), 'C': array([40.05907, 22.18248, ..., 33.62995,  2.73409])}},
       crossover_rate=0.1, cv=3, disable_update_check=False, early_stop=5,
       generations=3, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=5,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='neg_mean_squared_error', subsample=1.0,
       template=None, use_dask=False, verbosity=3, warm_start=False)

In [71]:
#KNN
from tpot import TPOTRegressor

parameters = {
    'n_neighbors': range(1,20),
             }
               
ga2 = TPOTRegressor(generations= 3, population_size= 10, offspring_size= 5,
                                 verbosity= 3, early_stop= 5,
                                 config_dict=
                                 {'sklearn.neighbors.KNeighborsRegressor': parameters}, 
                                 cv = 3, scoring = 'neg_mean_squared_error')
ga2.fit(X, y)

1 operators have been imported by TPOT.
Generation 1 - Current Pareto front scores:
-1	-80.83005201647829	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=6)

Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Generation 2 - Current Pareto front scores:
-1	-80.83005201647829	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=6)

Pipeline encountered that has previously been evaluated during the optimization process. Using the score from the previous evaluation.
Generation 3 - Current Pareto front scores:
-1	-80.83005201647829	KNeighborsRegressor(input_matrix, KNeighborsRegressor__n_neighbors=6)


The optimized pipeline was not improved after evaluating 5 more generations. Will end the optimization process.

TPOT closed prematurely. Will use the current best pipeline.


TPOTRegressor(config_dict={'sklearn.neighbors.KNeighborsRegressor': {'n_neighbors': range(1, 20)}},
       crossover_rate=0.1, cv=3, disable_update_check=False, early_stop=5,
       generations=3, max_eval_time_mins=5, max_time_mins=None,
       memory=None, mutation_rate=0.9, n_jobs=1, offspring_size=5,
       periodic_checkpoint_folder=None, population_size=10,
       random_state=None, scoring='neg_mean_squared_error', subsample=1.0,
       template=None, use_dask=False, verbosity=3, warm_start=False)